This notebook utilizes our data cleaning and preparations scripts to obtain clean, unlabeled text and serialize it in a .pkl file 

In [11]:
import pickle
import os 
from ietf_wg_mb_extractor import IETF_WG_MB_Extractor
from prepare_data import DataPreparator

In [3]:
archive_path = 'email-archives/'
extractor = IETF_WG_MB_Extractor(archive_path)
extractor.combine_wg_files(ratio=None)
active_wg_dataframe = extractor.get_combined_wg_dataframes()

Successfully scraped active WG names
------------------------------------
Successfully extracted names of active WGs existing in files
------------------------------------------------------------
Successfully converted and concatenated all .csv files into one dataframe
-------------------------------------------------------------------------


In [4]:
extractor = IETF_WG_MB_Extractor(archive_path)
data_preparator = DataPreparator(extractor)

clean_wg_dataframe = data_preparator.emails_df_cleaning(active_wg_dataframe)
processed_bodies = data_preparator.preprocess_bodies()
text_coll_combined = data_preparator.wg_combined_bodies_to_dict(processed_bodies)
text_coll_default = data_preparator.wg_bodies_to_dict(processed_bodies)

Initial dataframe shape: (499801, 8)
-----------------------------------------
Dataframe shape after cleaning rows: (46067, 9)

------PROCESSING------



100%|██████████| 46067/46067 [01:03<00:00, 725.56it/s] 


Prepocessing time: 63.50 s.

Number of mails removed: 12843
---------------------------------------
Encrypted messages: 3306
Ill from-formated messages: 208
Announc. messages: 4668
Unknown endcoding messages: 62
Empty messages post-processing: 4050
Diff. language: 462
Diverse other noise: 87


In [12]:
train_bodies = data_preparator.prepare_data_for_model(text_coll_default, seed=77, percent_of_data=0.99)

# Putting texts in dict, indexing and removing additional noise 
idx_texts = {i: body for i, body in  enumerate(train_bodies) if 'count bytes who' not in body}
# Serializing the unlabeled data
pickle.dump(idx_texts, open(os.path.join(".", "unlabeled_texts_dict.pkl"), "wb"))

Requested num. of samples: 30615
Actual num. of samples (due to post duplicate-removal): 30059
